In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_curve, RocCurveDisplay, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
import os

from sklearn.neighbors import KNeighborsClassifier


In [17]:
wisconsin = pd.read_csv('../Cases/Wisconsin/BreastCancer.csv', index_col = 0)
wisconsin

,Clump,UniCell_Size,Uni_CellShape,MargAdh,SEpith,BareN,BChromatin,NoemN,Mitoses,Class
Code,,,,,,,,,,
61634,5,4,3,1,2,2,2,3,1,Benign
63375,9,1,2,6,4,10,7,7,2,Malignant
76389,10,4,7,2,2,8,6,1,1,Malignant
95719,6,10,10,10,8,10,7,10,7,Malignant
128059,1,1,1,1,2,5,5,1,1,Benign
...,...,...,...,...,...,...,...,...,...,...
1369821,10,10,10,10,5,10,10,10,7,Malignant
1371026,5,10,10,10,4,10,5,6,3,Malignant
1371920,5,1,1,1,2,1,3,2,1,Benign


In [18]:
le = LabelEncoder()
wisconsin['Class'] = le.fit_transform(wisconsin['Class'])
x,y = wisconsin.drop('Class', axis = 1), wisconsin['Class']

In [19]:
wisconsin

,Clump,UniCell_Size,Uni_CellShape,MargAdh,SEpith,BareN,BChromatin,NoemN,Mitoses,Class
Code,,,,,,,,,,
61634,5,4,3,1,2,2,2,3,1,0
63375,9,1,2,6,4,10,7,7,2,1
76389,10,4,7,2,2,8,6,1,1,1
95719,6,10,10,10,8,10,7,10,7,1
128059,1,1,1,1,2,5,5,1,1,0
...,...,...,...,...,...,...,...,...,...,...
1369821,10,10,10,10,5,10,10,10,7,1
1371026,5,10,10,10,4,10,5,6,3,1
1371920,5,1,1,1,2,1,3,2,1,0


In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=25, stratify=y)

In [21]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)

y_pred = knn.predict(x_test)

print(f'accuracy_score: {accuracy_score(y_test,y_pred)}')

y_pred_proba = knn.predict_proba(x_test)

print(f'roc_auc_score: {roc_auc_score(y_test,y_pred_proba[:,1])}')

accuracy_score: 0.9761904761904762
roc_auc_score: 0.9908917069243157


calculating for diff neighbours

In [26]:
neighbours = [i for i in range(1,16)]
scores = []
for i in neighbours:
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_train, y_train)

    y_pred = knn.predict(x_test)
    y_pred_proba = knn.predict_proba(x_test)
    
    scores.append([i,accuracy_score(y_test,y_pred), roc_auc_score(y_test,y_pred_proba[:,1])])
    # print(f'accuracy_score: {accuracy_score(y_test,y_pred)}')
    # print(f'roc_auc_score: {roc_auc_score(y_test,y_pred_proba[:,1])}')
scores_df = pd.DataFrame(data = scores, columns = ['Neighbours', 'AccuracyScore', 'ROC_AUC'])
scores_df.sort_values('ROC_AUC', ascending=False)

,Neighbours,AccuracyScore,ROC_AUC
3,4,0.961905,0.991194
4,5,0.976190,0.990892
6,7,0.961905,0.990841
5,6,0.952381,0.990791
7,8,0.961905,0.990640
8,9,0.961905,0.990489
9,10,0.961905,0.990338
10,11,0.961905,0.990137
11,12,0.961905,0.989785
12,13,0.961905,0.989634


## Housing Dataset

In [27]:
hr = pd.read_csv('../datasets/HR_comma_sep.csv')
print(hr)

x,y = hr.drop('left', axis = 1), hr['left']

       satisfaction_level  last_evaluation  number_project  \
0                    0.38             0.53               2   
1                    0.80             0.86               5   
2                    0.10             0.77               6   
3                    0.92             0.85               5   
4                    0.89             1.00               5   
...                   ...              ...             ...   
14990                0.40             0.57               2   
14991                0.37             0.48               2   
14992                0.37             0.53               2   
14993                0.11             0.96               6   
14994                0.37             0.52               2   

       average_montly_hours  time_spend_company  Work_accident  left  \
0                       157                   3              0     1   
1                       262                   6              0     1   
2                       247            

In [29]:
ohe = OneHotEncoder(drop = 'first', sparse_output = False).set_output(transform = 'pandas')
col_trnf = ColumnTransformer([('OHE', ohe, make_column_selector(dtype_include = object))],
                             remainder = 'passthrough',
                             verbose_feature_names_out=False)
col_trnf = col_trnf.set_output(transform = 'pandas')
x = col_trnf.fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 25, stratify = y)


In [31]:
neighbours = [i for i in range(1,16)]
scores = []
for i in tqdm(neighbours):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_train, y_train)

    y_pred = knn.predict(x_test)
    y_pred_proba = knn.predict_proba(x_test)
    
    scores.append([i,accuracy_score(y_test,y_pred), roc_auc_score(y_test,y_pred_proba[:,1])])
    # print(f'accuracy_score: {accuracy_score(y_test,y_pred)}')
    # print(f'roc_auc_score: {roc_auc_score(y_test,y_pred_proba[:,1])}')
scores_df = pd.DataFrame(data = scores, columns = ['Neighbours', 'AccuracyScore', 'ROC_AUC'])
scores_df.sort_values('ROC_AUC', ascending=False)

100%|██████████| 15/15 [00:03<00:00,  4.63it/s]


,Neighbours,AccuracyScore,ROC_AUC
7,8,0.942209,0.974952
6,7,0.937986,0.974829
8,9,0.939542,0.974610
9,10,0.941320,0.974369
11,12,0.940653,0.974147
12,13,0.936430,0.974067
5,6,0.942654,0.974021
10,11,0.938208,0.973789
13,14,0.937097,0.973776
14,15,0.936653,0.973720
